In [16]:
import pandas as pd
import yfinance as yf
import sys
import os
sys.path.append(os.path.abspath("/Users/ananyaparikh/Documents/Coding/DeepRGARCH/DeepRGARCH"))
from utils import load_rv
# Load RV5 panel data from .pkl (AEX stocks usually)
rv_data = '/Users/ananyaparikh/Documents/Coding/DeepRGARCH/code submission/data/rv.pkl'

# Define stock split event (manual for now)
split_event = {
    'ticker': 'TSLA',
    'announcement': '2020-08-11',
    'effective': '2020-08-31',
    'ratio': '5-for-1'
}


In [17]:
# Add parent directory (the one containing 'rerech') to sys.path
sys.path.append(os.path.abspath("/Users/ananyaparikh/Documents/Coding/DeepRGARCH/DeepRGARCH"))
from DeepRGARCH.rerech.model import RealRECHD_2LSTM
from rerech import smc
from utils import realrech_prior, realrech_cstr_fn, load_rv


def run_split_model(rv_data, split_event):
    dataset = get_split_dataset(rv_data, split_event['ticker'], split_event['effective'])

    model = RealRECHD_2LSTM(data=(dataset['y_train'], dataset['y_test'], dataset['rv_train'], dataset['rv_test']))
    smc_obj = smc.SMCD(prior=realrech_2lstm_prior, cstr=realrech_cstr_fn, model=model)
    smc_obj.run()

    return smc_obj, dataset


In [20]:
def get_split_dataset(rv_data, ticker, effective_date):
    # Pull OHLCV data
    df_yf = yf.download(ticker, start="2018-01-01", end="2022-01-01")
    # Instead of checking for 'Adj Close'
    # Replace this block:
    if 'Adj Close' not in df_yf.columns:
        print(df_yf.head())
        raise ValueError("Downloaded data does not contain 'Adj Close'. Check ticker or date range.")

    # Do this instead:
    df_yf.columns = df_yf.columns.get_level_values(0)  # Flatten MultiIndex
    df_yf = df_yf[['Close']].rename(columns={'Close': 'close_price'})  # Use 'Close' column

    # Subset RV5 data for AEX (we’ll match dates only)
    rv_df = rv_data.xs('AEX', level=0).copy()
    matched_df = rv_df[['rv5']].copy()
    matched_df['close_price'] = df_yf['close_price']
    matched_df.dropna(inplace=True)  # Keep only common dates

    return get_dataset(matched_df)


In [21]:
smc_obj, dataset = run_split_model(rv_data, split_event)

[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low       Open     Volume
Ticker           TSLA       TSLA       TSLA       TSLA       TSLA
Date                                                             
2018-01-02  21.368668  21.474001  20.733334  20.799999   65283000
2018-01-03  21.150000  21.683332  21.036667  21.400000   67822500
2018-01-04  20.974667  21.236668  20.378668  20.858000  149194500
2018-01-05  21.105333  21.149332  20.799999  21.108000   68868000
2018-01-08  22.427334  22.468000  21.033333  21.066668  147891000


ValueError: Downloaded data does not contain 'Adj Close'. Check ticker or date range.